# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy.stats import percentileofscore as score

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

symbol = 'AAPL'
base_url = 'https://sandbox.iexapis.com/stable'
api_url = f"{base_url}/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
print(data)

{'companyName': 'Apple Inc', 'marketcap': 2557941099766, 'week52high': 152.47, 'week52low': 106.76, 'week52highSplitAdjustOnly': 157.3, 'week52lowSplitAdjustOnly': 106.3, 'week52change': 0.20699316731708478, 'sharesOutstanding': 17123671016, 'float': 0, 'avg10Volume': 81538375, 'avg30Volume': 74176904, 'day200MovingAvg': 136.74, 'day50MovingAvg': 151.58, 'employees': 151924, 'ttmEPS': 5.4, 'ttmDividendRate': 0.8858278899721885, 'dividendYield': 0.00578160040369104, 'nextDividendDate': '', 'exDividendDate': '2021-07-26', 'nextEarningsDate': '2021-10-15', 'peRatio': 28.232380898440802, 'beta': 1.51949977200874, 'maxChangePercent': 55.99237359447714, 'year5ChangePercent': 4.800702863031157, 'year2ChangePercent': 1.931200456730294, 'year1ChangePercent': 0.2681635208635533, 'ytdChangePercent': 0.12696637431251395, 'month6ChangePercent': 0.14786155706543416, 'month3ChangePercent': 0.17217081145949512, 'month1ChangePercent': 0.016139319198928553, 'day30ChangePercent': 0.021617925196975793, 'd

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.2681635208635533

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

# for symbol_string in symbol_strings:
#     print(symbol_string)

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
def momentum_data():
    dataframe = pd.DataFrame(columns = my_columns)

    for symbols in symbol_strings:
        # print(symbols)
        api_url = f"{base_url}/stock/market/batch?symbols={symbols}&types=stats,price&token={IEX_CLOUD_API_TOKEN}"
        data = requests.get(api_url).json()
        # print(data['AAPL'])
        # print(data['AAPL']['stats'])

        for symbol in symbols.split(','):
            price = data[symbol]['price']
            # year1ChangePercent = data[symbol]['stats']['year1ChangePercent'],
            buy_count = 'N/A'

            dataframe = dataframe.append(
                pd.Series(
                    [
                        symbol,
                        price,
                        data[symbol]['stats']['year1ChangePercent'],
                        buy_count
                    ],
                    index = my_columns
                ),
                ignore_index = True
            )

    return dataframe

final_dataframe = momentum_data()
print(final_dataframe)

    Ticker   Price One-Year Price Return Number of Shares to Buy
0        A  175.75              0.761571                     N/A
1      AAL   18.60                  0.49                     N/A
2      AAP  219.33              0.345372                     N/A
3     AAPL  151.20              0.271549                     N/A
4     ABBV  119.53              0.316757                     N/A
..     ...     ...                   ...                     ...
500    YUM  140.87              0.451386                     N/A
501    ZBH  146.13              0.072562                     N/A
502   ZBRA  580.26              1.069384                     N/A
503   ZION   54.82              0.753737                     N/A
504    ZTS  208.02              0.326186                     N/A

[505 rows x 4 columns]


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50] # Dump all but the first 50
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,83.66,2.327603,N/A
1,272,KSS,59.83,2.091276,N/A
2,441,TPR,41.86,1.886073,N/A
3,106,COF,173.76,1.653216,N/A
4,344,NUE,119.76,1.647149,N/A
5,129,DFS,128.64,1.610557,N/A
6,148,DVN,26.30,1.497418,N/A
7,253,IVZ,24.41,1.48208,N/A
8,23,ALB,225.60,1.463329,N/A
9,453,UAA,24.37,1.421598,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size

    portfolio_size = 1000000 # input('Enter the value of your portfolio:')

    try:
        float(portfolio_size)
    except ValueError:
        print('Portfolio size should be an integer.')
        portfolio_size = input('Enter the value of your portfolio:')
        float(portfolio_size)

portfolio_input()
print(portfolio_size)

1000000


In [9]:
position_size = float(portfolio_size) / len(final_dataframe.index)

for i in range(0, len(final_dataframe.index)):
    buy_count =  math.floor(position_size / final_dataframe.loc[i, 'Price'])
    final_dataframe.loc[i, 'Number of Shares to Buy'] = buy_count

final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,83.66,2.327603,239
1,272,KSS,59.83,2.091276,334
2,441,TPR,41.86,1.886073,477
3,106,COF,173.76,1.653216,115
4,344,NUE,119.76,1.647149,167
5,129,DFS,128.64,1.610557,155
6,148,DVN,26.30,1.497418,760
7,253,IVZ,24.41,1.48208,819
8,23,ALB,225.60,1.463329,88
9,453,UAA,24.37,1.421598,820


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [20]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
]

def hq_momentum_data():
    dataframe = pd.DataFrame(columns = my_columns)

    for symbols in symbol_strings:
        # print(symbols)
        api_url = f"{base_url}/stock/market/batch?symbols={symbols}&types=stats,price&token={IEX_CLOUD_API_TOKEN}"
        data = requests.get(api_url).json()
        # print(data['AAPL'])
        # print(data['AAPL']['stats'])

        for symbol in symbols.split(','):
            price = data[symbol]['price']
            # year1ChangePercent = data[symbol]['stats']['year1ChangePercent'],
            buy_count = np.nan

            dataframe = dataframe.append(
                pd.Series(
                    [
                        symbol,
                        price,
                        buy_count,
                        data[symbol]['stats']['year1ChangePercent'], 
                        np.nan,
                        data[symbol]['stats']['month6ChangePercent'], 
                        np.nan,
                        data[symbol]['stats']['month3ChangePercent'], 
                        np.nan,
                        data[symbol]['stats']['month1ChangePercent'], 
                        np.nan,
                    ],
                    index = hqm_columns,
                    
                ),
                ignore_index = True
            )

    # https://stackoverflow.com/questions/65174575/typeerror-not-supported-between-instances-of-nonetype-and-float
    return dataframe.fillna(0)

hqm_dataframe = hq_momentum_data()
hqm_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy,One-Month Price Return,One-Month Return Percentile,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile
0,A,169.30,0.731555,0.0,0.126634,0.0,0.0,0.343892,0.0,0.281225,0.0
1,AAL,19.00,0.480000,0.0,-0.102087,0.0,0.0,-0.010049,0.0,-0.189614,0.0
2,AAP,219.53,0.355022,0.0,-0.002706,0.0,0.0,0.280474,0.0,0.072028,0.0
3,AAPL,153.48,0.269088,0.0,0.015490,0.0,0.0,0.150905,0.0,0.167610,0.0
4,ABBV,123.18,0.311674,0.0,0.025049,0.0,0.0,0.159041,0.0,0.026091,0.0
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,137.66,0.438973,0.0,0.164729,0.0,0.0,0.298031,0.0,0.126386,0.0
501,ZBH,145.77,0.072247,0.0,-0.068557,0.0,0.0,-0.071053,0.0,-0.130081,0.0
502,ZBRA,597.18,1.082337,0.0,0.100412,0.0,0.0,0.165437,0.0,0.157188,0.0
503,ZION,54.67,0.739693,0.0,0.083116,0.0,0.0,0.056177,0.0,-0.050537,0.0


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [21]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month',
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        column_return = f'{time_period} Price Return'
        column_percentile = f'{time_period} Return Percentile'

        # print(hqm_dataframe.convert_dtypes().dtypes)
        # print(hqm_dataframe[column_return])
        price_return_all = pd.to_numeric(hqm_dataframe[column_return].astype(float))
        price_return_single = hqm_dataframe.loc[row, column_return]

        percentile = score(price_return_all, price_return_single)
        hqm_dataframe.loc[row, column_percentile] = percentile

hqm_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy,One-Month Price Return,One-Month Return Percentile,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile
0,A,169.30,0.731555,0.0,0.126634,93.465347,84.356436,0.343892,89.306931,0.281225,95.841584
1,AAL,19.00,0.480000,0.0,-0.102087,4.356436,63.564356,-0.010049,17.821782,-0.189614,3.960396
2,AAP,219.53,0.355022,0.0,-0.002706,30.891089,45.742574,0.280474,80.198020,0.072028,66.336634
3,AAPL,153.48,0.269088,0.0,0.015490,42.574257,34.653465,0.150905,54.455446,0.167610,85.940594
4,ABBV,123.18,0.311674,0.0,0.025049,47.326733,40.594059,0.159041,57.029703,0.026091,51.881188
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,137.66,0.438973,0.0,0.164729,97.029703,58.811881,0.298031,82.772277,0.126386,78.217822
501,ZBH,145.77,0.072247,0.0,-0.068557,10.495050,13.465347,-0.071053,10.099010,-0.130081,10.297030
502,ZBRA,597.18,1.082337,0.0,0.100412,87.722772,95.445545,0.165437,59.207921,0.157188,83.762376
503,ZION,54.67,0.739693,0.0,0.083116,82.574257,84.950495,0.056177,33.069307,-0.050537,29.306931


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: